In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# import imageio
import random
cv2.ocl.setUseOpenCL(False)
import warnings
warnings.filterwarnings('ignore')

# from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import matplotlib.pyplot as plt

In [15]:
import cv2
import numpy as np
import os
import re 
from exceptions import *
import plotly.graph_objects as go
from plotly.subplots import make_subplots

MINIMUM_MATCH_POINTS = 15 #The minimum number of matched keypoints required to consider the matching process successfu
CONFIDENCE_THRESH = 10 # confidence percentage threshold of match points used for homography computation



def plot_interactive_keypoints(image, keypoints, title):
    fig = go.Figure(data=[go.Scatter(x=[kp[0] for kp in keypoints], y=[kp[1] for kp in keypoints],
                                     mode='markers', marker=dict(size=5, color='purple'))])
    fig.update_layout(title=title, xaxis_title='X Coordinate', yaxis_title='Y Coordinate',
                      width=800, height=600, autosize=False)
    fig.update_xaxes(range=[0, image.shape[1]])
    fig.update_yaxes(range=[0, image.shape[0]], autorange="reversed")  # Image coordinates are reversed in y-axis
    fig.show()

def draw_keypoints(vis, keypoints, color):
    for kp in keypoints:
        x, y = kp.pt
        cv2.circle(vis, (int(x), int(y)), 5, color, -1)





def get_matches(img_a_gray, img_b_gray, num_keypoints=1000, threshold=0.8):
    '''Function to get matched keypoints from two images using ORB

    Args:
        img_a_gray (numpy array): of shape (H, W) representing grayscale image A
        img_b_gray (numpy array): of shape (H, W) representing grayscale image B
        num_keypoints (int): number of points to be matched (default=100)
        threshold (float): can be used to filter strong matches only. 
        Lower the value, stronger the requirements and hence fewer matches.
        
        A lower threshold value indicates stronger matches because it requires the closest match 
        (first nearest neighbor) to be significantly closer than the second nearest neighbor.
        This large disparity ensures that the closest 
        match is much more likely to be correct, reducing the chance of false matches.
    Returns:
        match_points_a (numpy array): of shape (n, 2) representing x,y pixel coordinates of image A keypoints
        match_points_b (numpy array): of shape (n, 2) representing x,y pixel coordianted of matched keypoints in image B
    '''
    orb = cv2.ORB_create(nfeatures=num_keypoints)
    # find keypoints and their descriptors
    kp_a, desc_a = orb.detectAndCompute(img_a_gray, None)
    kp_b, desc_b = orb.detectAndCompute(img_b_gray, None)
    
    #brute force matcher using the hamming distance, gets closest matches
    dis_matcher = cv2.BFMatcher(cv2.NORM_HAMMING)
    # specifically retrieves the k nearest neighbors for each descriptor
    matches_list = dis_matcher.knnMatch(desc_a, desc_b, k=2) # get the two nearest matches for each keypoint in image A

    # for each keypoint feature in image A, compare the distance of the two matched keypoints in image B
    # retain only if distance is less than a threshold 
    good_matches_list = []
    for match_1, match_2 in matches_list:
        # if the distance of the closer match (match_1) is less than threshold * distance
        #  of the farther match (match_2), match_1 is considered a good match.
        if match_1.distance < threshold * match_2.distance:
            good_matches_list.append(match_1)
    
    #filter good matching keypoints 
    good_kp_a = []
    good_kp_b = []
    
    # For each match, retrieve the coordinate of the keypoint in image A (good_kp_a) using queryIdx and 
    # in image B (good_kp_b) using trainIdx, 
    # ensuring that these keypoints correspond to each other across the two images.
    for match in good_matches_list:
        # .pt is an attribute of a keypoint object returned by feature detection methods like ORB. The .pt attribute
        # contains the (x, y) coordinates of the keypoint in the image. 
        good_kp_a.append(kp_a[match.queryIdx].pt) # keypoint in image A
        good_kp_b.append(kp_b[match.trainIdx].pt) # matching keypoint in image B

    img_kp_a = cv2.drawKeypoints(img_a_gray, kp_a, None, color=(0, 255, 0), flags=0)
    img_kp_b = cv2.drawKeypoints(img_b_gray, kp_b, None, color=(0, 255, 0), flags=0)

    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    plt.imshow(img_kp_a, cmap='gray')
    plt.title('Keypoints in Image A')
    plt.axis('off')

    plt.subplot(122)
    plt.imshow(img_kp_b, cmap='gray')
    plt.title('Keypoints in Image B')
    plt.axis('off')

    plt.show()

    # Visualization of keypoints and matches
    img_matches = cv2.drawMatches(img_a_gray, kp_a, img_b_gray, kp_b, good_matches_list, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.figure(figsize=(12, 6))
    plt.imshow(img_matches)
    plt.title('Key Points and Matches')
    plt.axis('off')
    plt.show()
    
    if len(good_kp_a) < MINIMUM_MATCH_POINTS:
        raise NotEnoughMatchPointsError(len(good_kp_a), MINIMUM_MATCH_POINTS)
    
    return np.array(good_kp_a), np.array(good_kp_b)


def calculate_homography(points_img_a, points_img_b):
    '''Function to calculate the homography matrix from point corresspondences using Direct Linear Transformation
        The resultant homography transforms points in image B into points in image A
        Homography H = [h1 h2 h3; 
                        h4 h5 h6;
                        h7 h8 h9]
        u, v ---> point in image A
        x, y ---> matched point in image B then,
        with n point correspondences the DLT equation is:
            A.h = 0
        where A = [-x1 -y1 -1 0 0 0 u1*x1 u1*y1 u1;
                   0 0 0 -x1 -y1 -1 v1*x1 v1*y1 v1;
                   ...............................;
                   ...............................;
                   -xn -yn -1 0 0 0 un*xn un*yn un;
                   0 0 0 -xn -yn -1 vn*xn vn*yn vn]
        This equation is then solved using SVD
        (At least 4 point correspondences are required to determine 8 unkwown parameters of homography matrix)
    Args:
        points_img_a (numpy array): of shape (n, 2) representing pixel coordinate points (u, v) in image A
        points_img_b (numpy array): of shape (n, 2) representing pixel coordinates (x, y) in image B
    
    Returns:
        h_mat: A (3, 3) numpy array of estimated homography
    '''
    # concatenate the two numpy points array to get 4 columns (u, v, x, y)
    points_a_and_b = np.concatenate((points_img_a, points_img_b), axis=1)
    A = []
    # fill the A matrix by looping through each row of points_a_and_b containing u, v, x, y
    # each row in the points_ab would fill two rows in the A matrix
    for u, v, x, y in points_a_and_b:
        A.append([-x, -y, -1, 0, 0, 0, u*x, u*y, u])
        A.append([0, 0, 0, -x, -y, -1, v*x, v*y, v])
    
    A = np.array(A)
    _, _, v_t = np.linalg.svd(A)

    # soltion is the last column of v which means the last row of its transpose v_t
    h_mat = v_t[-1, :].reshape(3,3)
    return h_mat

def transform_with_homography(h_mat, points_array):
    """Function to transform a set of points using the given homography matrix.
        Points are normalized after transformation with the last column which represents the scale
    
    Args:
        h_mat (numpy array): of shape (3, 3) representing the homography matrix
        points_array (numpy array): of shape (n, 2) represting n set of x, y pixel coordinates that are
            to be transformed
    """
    # add column of ones so that matrix multiplication with homography matrix is possible
    ones_col = np.ones((points_array.shape[0], 1))
    # axis 1 means horizontally a new column of ones is added
    points_array = np.concatenate((points_array, ones_col), axis=1)

    # multiply the homography matrix to get the transformed points from points array
    transformed_points = np.matmul(h_mat, points_array.T)
    epsilon = 1e-7 # very small value to use it during normalization to avoid division by zero

# Access the third row which contains the scale 
# factors (w) for each point, is accessed.
# reshaped into a 2D array with one row to allow for element-wise division 
# Epsilon to prevent division by zero in case any scale factor is zero.
# All coordinates  are divided by these adjusted scale factors,
#  converting points from homogeneous coordinates (x, y, w) back to 
# Cartesian coordinates (x/w, y/w), effectively normalizing 
    transformed_points = transformed_points / (transformed_points[2,:].reshape(1,-1) + epsilon)
    transformed_points = transformed_points[0:2,:].T
    return transformed_points


def compute_outliers(h_mat, points_img_a, points_img_b, threshold=3):
    '''Function to compute the error in the Homography matrix using the matching points in
        image A and image B
    
    Args:
        h_mat (numpy array): of shape (3, 3) representing the homography that transforms points in image B to points in image A
        points_img_a (numpy array): of shape (n, 2) representing pixel coordinate points (u, v) in image A
        points_img_b (numpy array): of shape (n, 2) representing pixel coordinates (x, y) in image B
        theshold (int): a number that represents the allowable euclidean distance (in pixels) between the transformed pixel coordinate from
            the image B to the matched pixel coordinate in image A, to be conisdered outliers
    
    Returns:
        error: a scalar float representing the error in the Homography matrix
    '''
    num_points = points_img_a.shape[0]
    outliers_count = 0

    # transform the match point in image B to image A using the homography
    points_img_b_hat = transform_with_homography(h_mat, points_img_b)
    
    # let x, y be coordinate representation of points in image A
    # let x_hat, y_hat be the coordinate representation of transformed points of image B with respect to image A
    x = points_img_a[:, 0]
    y = points_img_a[:, 1]
    x_hat = points_img_b_hat[:, 0]
    y_hat = points_img_b_hat[:, 1]
    euclid_dis = np.sqrt(np.power((x_hat - x), 2) + np.power((y_hat - y), 2)).reshape(-1)
    for dis in euclid_dis:
        if dis > threshold:
            outliers_count += 1
    return outliers_count


# def compute_homography_ransac(matches_a, matches_b):
#     """Function to estimate the best homography matrix using RANSAC on potentially matching
#     points.
    
#     Args:
#         matches_a (numpy array): of shape (n, 2) representing the coordinates
#             of possibly matching points in image A
#         matches_b (numpy array): of shape (n, 2) representing the coordinates
#             of possibly matching points in image B

#     Returns:
#         best_h_mat: A numpy array of shape (3, 3) representing the best homography
#             matrix that transforms points in image B to points in image A
#     """
#     num_all_matches =  matches_a.shape[0]
#     # RANSAC parameters
#     SAMPLE_SIZE = 5 #number of point correspondances for estimation of Homgraphy
#     SUCCESS_PROB = 0.995 #required probabilty of finding H with all samples being inliners 
#     min_iterations = int(np.log(1.0 - SUCCESS_PROB)/np.log(1 - 0.5**SAMPLE_SIZE))
    
#     # Let the initial error be large i.e consider all matched points as outliers
#     lowest_outliers_count = num_all_matches
#     best_h_mat = None
#     best_i = 0 # just to know in which iteration the best h_mat was found

#     for i in range(min_iterations):
#         rand_ind = np.random.permutation(range(num_all_matches))[:SAMPLE_SIZE]
#         h_mat = calculate_homography(matches_a[rand_ind], matches_b[rand_ind])
#         outliers_count = compute_outliers(h_mat, matches_a, matches_b)
#         if outliers_count < lowest_outliers_count:
#             best_h_mat = h_mat
#             lowest_outliers_count = outliers_count
#             best_i = i
#     best_confidence_obtained = int(100 - (100 * lowest_outliers_count / num_all_matches))
#     if best_confidence_obtained < CONFIDENCE_THRESH:
#         raise(exceptions.MatchesNotConfident(best_confidence_obtained))
#     return best_h_mat

def compute_homography_ransac(matches_a, matches_b):
    """Function to estimate the best homography matrix using RANSAC on potentially matching
    points.
    
    Args:
        matches_a (numpy array): of shape (n, 2) representing the coordinates
            of possibly matching points in image A
        matches_b (numpy array): of shape (n, 2) representing the coordinates
            of possibly matching points in image B

    Returns:
        best_h_mat: A numpy array of shape (3, 3) representing the best homography
            matrix that transforms points in image B to points in image A
    """

    # def compute_homography_ransac(matches_a, m
    print("Running modified compute_homography_ransac")
    # existing implementation

    num_all_matches = matches_a.shape[0]
    SAMPLE_SIZE = 5
    SUCCESS_PROB = 0.995
    # RANSAC algorithm's guidelines to calculate the likelihood of success over several iterations.
    min_iterations = int(np.log(1.0 - SUCCESS_PROB) / np.log(1 - 0.5**SAMPLE_SIZE))

    lowest_outliers_count = num_all_matches
    best_h_mat = None

    for i in range(min_iterations):
        # select random set of points of sample size
        rand_ind = np.random.permutation(range(num_all_matches))[:SAMPLE_SIZE]
        # calculate homography matrix based on this
        h_mat = calculate_homography(matches_a[rand_ind], matches_b[rand_ind])
        # then given the matrix and the matches of both images, count the outliers
        outliers_count = compute_outliers(h_mat, matches_a, matches_b)
        # if it is lower make this the best homography matrix
        if outliers_count < lowest_outliers_count:
            best_h_mat = h_mat
            lowest_outliers_count = outliers_count

    # calculates the percentage of matches that are outliers.
    # subtracting from 100 gives inliers percentage 
    best_confidence_obtained = int(100 - (100 * lowest_outliers_count / num_all_matches))
    print("confidence obtained in ransac", best_confidence_obtained)
    if best_confidence_obtained < CONFIDENCE_THRESH:
        raise(exceptions.MatchesNotConfident(best_confidence_obtained, CONFIDENCE_THRESH))
    return best_h_mat

def get_corners_as_array(img_height, img_width):
    """Function to extract the corner points of an image from its width and height 
        and arrange it in the form
        of a numpy array.
        
        The 4 corners are arranged as follows:
        corners = [top_left_x, top_left_y;
                   top_right_x, top_right_y;
                   bottom_right_x, bottom_right_y;
                   bottom_left_x, bottom_left_y]

    Args:
        img_height (str): height of the image
        img_width (str): width of the image
    
    Returns:
        corner_points_array (numpy array): of shape (4,2) representing for corners with x,y pixel coordinates
    """
    corners_array = np.array([[0, 0],
                            [img_width - 1, 0],
                            [img_width - 1, img_height - 1],
                            [0, img_height - 1]])
    return corners_array


def get_crop_points_horz(img_a_h, transfmd_corners_img_b):
    """Function to find the pixel corners in the horizontally stitched images to crop and remove the
        black space around.
    
    Args:   
        img_a_h (int): the height of the pivot image that is image A
        transfmd_corners_img_b (numpy array): of shape (n, 2) representing the transformed corners of image B
            The corners need to be in the following sequence:
            corners = [top_left_x, top_left_y;
                   top_right_x, top_right_y;
                   bottom_right_x, bottom_right_y;
                   bottom_left_x, bottom_left_y]
    Returns:
        x_start (int): the x pixel-cordinate to start the crop on the stitched image
        y_start (int): the x pixel-cordinate to start the crop on the stitched image
        x_end (int): the x pixel-cordinate to end the crop on the stitched image
        y_end (int): the y pixel-cordinate to end the crop on the stitched image
    """
    # the four transformed corners of image B
    top_lft_x_hat, top_lft_y_hat = transfmd_corners_img_b[0, :]
    top_rht_x_hat, top_rht_y_hat = transfmd_corners_img_b[1, :]
    btm_rht_x_hat, btm_rht_y_hat = transfmd_corners_img_b[2, :]
    btm_lft_x_hat, btm_lft_y_hat = transfmd_corners_img_b[3, :]

    # initialize the crop points
    # since image A (on the left side) is used as pivot, x_start will always be zero
    x_start, y_start, x_end, y_end = (0, None, None, None)

    if (top_lft_y_hat > 0) and (top_lft_y_hat > top_rht_y_hat):
        y_start = top_lft_y_hat
    elif (top_rht_y_hat > 0) and (top_rht_y_hat > top_lft_y_hat):
        y_start = top_rht_y_hat
    else:
        y_start = 0
        
    if (btm_lft_y_hat < img_a_h - 1) and (btm_lft_y_hat < btm_rht_y_hat):
        y_end = btm_lft_y_hat
    elif (btm_rht_y_hat < img_a_h - 1) and (btm_rht_y_hat < btm_lft_y_hat):
        y_end = btm_rht_y_hat
    else:
        y_end = img_a_h - 1

    if (top_rht_x_hat < btm_rht_x_hat):
        x_end = top_rht_x_hat
    else:
        x_end = btm_rht_x_hat
    
    return int(x_start), int(y_start), int(x_end), int(y_end)


def get_crop_points_vert(img_a_w, transfmd_corners_img_b):
    """Function to find the pixel corners in the vertically stitched images to crop and remove the
        black space around.
    
    Args:
        img_a_h (int): the width of the pivot image that is image A
        transfmd_corners_img_b (numpy array): of shape (n, 2) representing the transformed corners of image B
            The corners need to be in the following sequence:
            corners = [top_left_x, top_left_y;
                   top_right_x, top_right_y;
                   bottom_right_x, bottom_right_y;
                   bottom_left_x, bottom_left_y]
    Returns:
        x_start (int): the x pixel-cordinate to start the crop on the stitched image
        y_start (int): the x pixel-cordinate to start the crop on the stitched image
        x_end (int): the x pixel-cordinate to end the crop on the stitched image
        y_end (int): the y pixel-cordinate to end the crop on the stitched image
    """
    # the four transformed corners of image B
    top_lft_x_hat, top_lft_y_hat = transfmd_corners_img_b[0, :]
    top_rht_x_hat, top_rht_y_hat = transfmd_corners_img_b[1, :]
    btm_rht_x_hat, btm_rht_y_hat = transfmd_corners_img_b[2, :]
    btm_lft_x_hat, btm_lft_y_hat = transfmd_corners_img_b[3, :]

    # initialize the crop points
    # since image A (on the top) is used as pivot, y_start will always be zero
    x_start, y_start, x_end, y_end = (None, 0, None, None)

    if (top_lft_x_hat > 0) and (top_lft_x_hat > btm_lft_x_hat):
        x_start = top_lft_x_hat
    elif (btm_lft_x_hat > 0) and (btm_lft_x_hat > top_lft_x_hat):
        x_start = btm_lft_x_hat
    else:
        x_start = 0
        
    if (top_rht_x_hat < img_a_w - 1) and (top_rht_x_hat < btm_rht_x_hat):
        x_end = top_rht_x_hat
    elif (btm_rht_x_hat < img_a_w - 1) and (btm_rht_x_hat < top_rht_x_hat):
        x_end = btm_rht_x_hat
    else:
        x_end = img_a_w - 1

    if (btm_lft_y_hat < btm_rht_y_hat):
        y_end = btm_lft_y_hat
    else:
        y_end = btm_rht_y_hat
    
    return int(x_start), int(y_start), int(x_end), int(y_end)


def get_crop_points(h_mat, img_a, img_b, stitch_direc):
    """Function to find the pixel corners to crop the stitched image such that the black space 
        in the stitched image is removed.
        The black space could be because either image B is not of the same dimensions as image A
        or image B is skewed after homographic transformation.
        Example: 
                  (Horizontal stitching)
                ____________                     _________________
                |           |                    |                |
                |           |__________          |                |
                |           |         /          |       A        |
                |     A     |   B    /           |________________|
                |           |       /                |          | 
                |           |______/                 |    B     |
                |___________|                        |          |
                                                     |__________|  <-imagine slant bottom edge
        
        This function returns the corner points to obtain the maximum area inside A and B combined and making
        sure the edges are straight (i.e horizontal and veritcal). 

    Args:
        h_mat (numpy array): of shape (3, 3) representing the homography from image B to image A
        img_a (numpy array): of shape (h, w, c) representing image A
        img_b (numpy array): of shape (h, w, c) representing image B
        stitch_direc (int): 0 when stitching vertically and 1 when stitching horizontally

    Returns:
        x_start (int): the x pixel-cordinate to start the crop on the stitched image
        y_start (int): the x pixel-cordinate to start the crop on the stitched image
        x_end (int): the x pixel-cordinate to end the crop on the stitched image
        y_end (int): the y pixel-cordinate to end the crop on the stitched image          
    """

    # extract height and width of the images
    img_a_h, img_a_w, _ = img_a.shape
    img_b_h, img_b_w, _ = img_b.shape

    # corner points of image b
    orig_corners_img_b = get_corners_as_array(img_b_h, img_b_w)

    # applies homography and returns transformed corners of the image b 
    transfmd_corners_img_b = transform_with_homography(h_mat, orig_corners_img_b)

    # if stitch_direc == 1:
    #     x_start, y_start, x_end, y_end = get_crop_points_horz(img_a_w, transfmd_corners_img_b)
    # initialize the crop points
    x_start = None
    x_end = None
    y_start = None
    y_end = None

    if stitch_direc == 1: # 1 is horizontal
        x_start, y_start, x_end, y_end = get_crop_points_horz(img_a_h, transfmd_corners_img_b)
    else: # when stitching images in the vertical direction
        x_start, y_start, x_end, y_end = get_crop_points_vert(img_a_w, transfmd_corners_img_b)
    return x_start, y_start, x_end, y_end


def stitch_image_pair(img_a, img_b, stitch_direc):
    print("inside stich image pair function")

    """Function to stitch image B to image A in the mentioned direction

    Args:
        img_a (numpy array): of shape (H, W, C) with opencv representation of image A (i.e C: B,G,R)
        img_b (numpy array): of shape (H, W, C) with opencv representation of image B (i.e C: B,G,R)
        stitch_direc (int): 0 for vertical and 1 for horizontal stitching

    Returns:
        stitched_image (numpy array): stitched image with maximum content of image A and image B after cropping
            to remove the black space 
    """

    print("inside stich image pair function")
    img_a_gray = cv2.cvtColor(img_a, cv2.COLOR_BGR2GRAY)
    img_b_gray = cv2.cvtColor(img_b, cv2.COLOR_BGR2GRAY)
    matches_a, matches_b = get_matches(img_a_gray, img_b_gray, num_keypoints=1000, threshold=0.8)

    print("matching done")

    h_mat = compute_homography_ransac(matches_a, matches_b)

    print("homography done")

    # on image b: img_b the transformation is done
    # h_matrix defines the necessary transformation required to align b with image a

    if stitch_direc == 0:
        # width same as image a, height is the combined image height of a and b    
        canvas = cv2.warpPerspective(img_b, h_mat, (img_a.shape[1], img_a.shape[0] + img_b.shape[0]))
        # now the canvas has the warped image b
        # now overlay image a in the portion of canvas where it is required
        # aligning with top edge
        canvas[0:img_a.shape[0], :, :] = img_a[:, :, :]
        # get the points of canvas where image overlap properly so, to exclude
        # or crop the remaining parts of image
        x_start, y_start, x_end, y_end = get_crop_points(h_mat, img_a, img_b, 0)
    else:
        canvas = cv2.warpPerspective(img_b, h_mat, (img_a.shape[1] + img_b.shape[1], img_a.shape[0]))
        canvas[:, 0:img_a.shape[1], :] = img_a[:, :, :]
        x_start, y_start, x_end, y_end = get_crop_points(h_mat, img_a, img_b, 1)
    
    stitched_img = canvas[y_start:y_end,x_start:x_end,:]
    return stitched_img


def check_imgfile_validity(folder, filenames):
    """Function to check if the files in the given path are valid image files.
    
    Args:
        folder (str): path containing the image files
        filenames (list): a list of image filenames

    Returns:
        valid_files (bool): True if all the files are valid image files else False
        msg (str): Message that has to be displayed as error
    """
    for file in filenames:
        full_file_path = os.path.join(folder, file)
        regex = "([^\\s]+(\\.(?i:(jpe?g|png)))$)"
        p = re.compile(regex)

        if not os.path.isfile(full_file_path):
            return False, "File not found: " + full_file_path
        if not (re.search(p, file)):
            return False, "Invalid image file: " + file
    return True, None


In [16]:
from utils import *
from exceptions import *
import os
import cv2
import time

def stitch_images(image_folder, image_filenames, stitch_direction):
    """Function to stitch a sequence of input images.
        Images can be stitched horizontally or vertically.
        For horizontal stitching the images have to be passed from left to right order in the scene.
        For vertical stitching the images have to be passed from top to bottom order in the scene.
    
    Args:
        image_folder (str): path of the directory containing the images
        image_filenames (list): a list of image file names in the order of stitching
        stitch_direction (int): 1 for horizontal stitching, 0 for vertical stitching
    
    Returns:
        stitched_image (numpy array): of shape (H, W, 3) representing the stitched image
    """
   
    num_images = len(image_filenames)
    
    #if number of images is less than 2 raise error
    if num_images < 2:
        raise(exceptions.InsufficientImagesError(num_images))
    
    # check for files validity
    valid_files, file_error_msg = check_imgfile_validity(image_folder, image_filenames)
    if not valid_files:
        raise(exceptions.InvalidImageFilesError(file_error_msg))
    
    print("valid files done")
    #takes first image as the pivot image
    # In panorama stitching, the pivot image is the central reference 
    # image around which other images
    # are aligned and stitched to create a seamless panoramic view.
    pivot_img_path = os.path.join(image_folder, image_filenames[0])
    pivot_img = cv2.imread(pivot_img_path)
    print("read pivot image")

    # Loop through all images starting from the second image in the list
    for i in range(1, num_images, 1):
            # Construct the full path for each image and read it.
        join_img_path = os.path.join(image_folder, image_filenames[i])
        join_img = cv2.imread(join_img_path)
        try:
            print("About to start stitching pairs")
            print("i is ", i)
            #stitch this image with pivot image
            pivot_img = stitch_image_pair(pivot_img, join_img, stitch_direc=stitch_direction)
            print("Finished stitching pairs")
        except Exception as e:
            print(f"An error occurred: {e}")
            
    return pivot_img

def stitch_images_and_save(image_folder, image_filenames, stitch_direction, output_folder=None):
    """Function to stitch and save the resultant image.
        Images can be stitched horizontally or vertically.
        For horizontal stitching the images have to be passed from left to right order in the scene.
        For vertical stitching the images have to be passed from top to bottom order in the scene.
    
    Args:
        image_folder (str): path of the directory containing the images
        image_filenames (list): a list of image file names in the order of stitching
        stitch_direction (int): 1 for horizontal stitching, 0 for vertical stitching
        output_folder (str): the directory to save the stitched image (default is None, which creates a directory named "output" to save)
    
    Returns:
        None
    """
    timestr = time.strftime("%Y%m%d_%H%M%S")
    filename = "stitched_image_" + timestr + ".jpg"
    stitched_img = stitch_images(image_folder, image_filenames, stitch_direction)
    if output_folder is None:
        if not os.path.isdir("output"):
            os.makedirs("output/")
        output_folder = "output"
    full_save_path = os.path.join(output_folder, filename)
    _ = cv2.imwrite(full_save_path, stitched_img)
    print("The stitched image is saved at: " + full_save_path)

In [17]:
image_folder = "img_folder"
# image_filenames=["scene1_a.jpg", "scene1_b.jpg", "scene1_c.jpg"]
# image_filenames=["scene2_a.jpg", "scene2_b.jpg", "scene2_c.jpg"]
image_filenames=["image1.jpg", "image2.jpg"]
# image_filenames=["badal1.jpeg", "badal2.jpeg"]
# image_filenames=["room1.jpeg", "room2.jpeg", "room3.jpeg"]
# image_filenames=["r1.jpeg", "r2.jpeg"]

print("Hewllo World ")
stitch_direction=1
#output_folder="./img_output"

#stitch_images(image_folder, image_filenames, stitch_direction)
stitch_images_and_save(image_folder, image_filenames, stitch_direction)

Hewllo World 
valid files done
read pivot image
About to start stitching pairs
i is  1
inside stich image pair function
inside stich image pair function
matching done
Running modified compute_homography_ransac
Heloooooo 74
homography done
Finished stitching pairs
The stitched image is saved at: output/stitched_image_20240507_125805.jpg
